# Deep Learning Models etc

In [4]:
from IPython.display import HTML
import import_ipynb
import random
import utils
utils.hide_toggle('Imports 1')

importing Jupyter notebook from utils.ipynb


In [5]:
import numpy as np
import torch
from torch import nn
from torch import optim
from IPython import display
utils.hide_toggle('Imports 2')

Compute accuracy of predictions.

In [40]:
def accuracy(Net,X_test,y_test,verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy
utils.hide_toggle('Function: accuracy')

Generic training loop

In [7]:
def Train(Net,data,epochs=20,lr=5e-2,Loss=nn.NLLLoss(),verbose=False):
    #optimizer = optim.Adam(Net.parameters(),lr=lr)
    losses = []
    accs = []
    for e in range(epochs):
        step=0
        tot_loss=0.0
        acc=0.0
        for (X,y) in data:
            y_pred = Net(X)
            loss = Loss(y_pred,y)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
            acc+=accuracy(Net,X,y,verbose=False)
        l = tot_loss.item()/step
        a = acc/step
        losses += [l]
        accs += [a]
        if verbose: 
            print('Epoch  % 2d Loss: %2.5e Accuracy: %2.5f'%(e,l,a))
            display.clear_output(wait=True)
    return Net,losses,accs
utils.hide_toggle('Function Train')

Multi-layer perceptron with ReLU non-lineartities; for classification or regression.

In [11]:
class MLP(nn.Module):
    def __init__(self,dims=[5,3,2],task='classification',lr=1e-3):
        super(MLP,self).__init__()
        self.dims=dims
        self.n = len(self.dims)-1
        self.task=task
        self.layers=nn.ModuleList()
        for i in range(self.n-1):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())
        if task=='classification': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.LogSoftmax(dim=1))
        elif task=='regression': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.Linear(dims[i+2],1))
        else: self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self,x):
        for l in self.layers:
            x = l(x)
        return(x)
utils.hide_toggle('Class MLP')

Recurrent network using GRU

In [9]:
class RNN(nn.Module):
    def __init__(self,n_features=1,dim=5,n_layers=3,n_classes=2,lr=1e-3):
        super(RNN,self).__init__()
        self.n_layers = n_layers
        self.dim = dim
        self.n_features = n_features
        self.rnn = nn.GRU(n_features, dim, n_layers,batch_first=True)
        self.linear = nn.Linear(dim,n_classes)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self,X):
        if self.n_features==1: X=X.unsqueeze(-1)
        h = torch.randn(self.n_layers,X.shape[0],self.dim)
        H = self.rnn(X,h)[1]
        G = self.logsoft(self.linear(H[-1,:,:]))
        return G
utils.hide_toggle('Class RNN')